In [4]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [5]:
import h5py as h5
filename = 'SNII.hdf5'
# Read H5 file
f = h5.File(filename, "r")
# Get and print list of datasets within the H5 file
datasetNames = [n for n in f.keys()]
for n in datasetNames:
    print(n)

Masses
Metallicities
Number_of_masses
Number_of_metallicities
Number_of_species
Reference
Species_names
Yield_names
Yields


In [66]:
Masses = f['Masses'][:]

In [67]:
Masses

array([   8.,    9.,   12.,   13.,   15.,   18.,   20.,   25.,   30.,
         40.,   60.,  100.,  120.], dtype=float32)

In [68]:
Metallicities = f['Metallicities'][:]

In [69]:
Metallicities

array([ 0.   ,  0.001,  0.004,  0.02 ], dtype=float32)

In [70]:
elements = f['Species_names'][:]

In [71]:
elements

array([b'Hydrogen', b'Helium', b'Carbon', b'Nitrogen', b'Oxygen', b'Neon',
       b'Magnesium', b'Silicon', b'Sulphur', b'Calcium', b'Iron'], dtype=object)

In [72]:
f['Yield_names'][:]

array([b'Z_0.00', b'Z_0.001', b'Z_0.004', b'Z_0.02'], dtype=object)

In [75]:
f['Yields/Z_0.001/Ejected_mass'][:]

array([  4.89106178,   5.70875597,   8.1308527 ,   8.69053936,
        10.25411034,  12.44572258,  14.89804363,  19.54817581,
        24.06858063,  31.77251053,  30.46581078,  51.58944321,  65.626091  ], dtype=float32)

In [76]:
f['Yields/Z_0.001/Yield'][:]

array([[ -7.79314339e-01,  -1.09488809e+00,  -2.09181118e+00,
         -1.88346457e+00,  -2.33148146e+00,  -3.21717787e+00,
         -5.03216553e+00,  -7.40000820e+00,  -9.68176746e+00,
         -1.35367889e+01,  -9.20366573e+00,  -1.87171211e+01,
         -2.64444199e+01],
       [  6.58688664e-01,   7.08719432e-01,   7.67138720e-01,
          9.03589010e-01,   1.53014314e+00,   2.08124375e+00,
          1.05877852e+00,   7.95940995e-01,   8.68318021e-01,
          1.19272101e+00,   4.31708527e+00,   7.75668573e+00,
          2.19261875e+01],
       [  2.45482605e-02,   5.08369617e-02,   9.85278487e-02,
          5.86317927e-02,   4.36407141e-02,   6.95605502e-02,
          6.62355274e-02,   1.16094023e-01,   5.43745980e-02,
          1.93091854e-02,   2.14126989e-01,   2.22935289e-01,
          2.12273151e-01],
       [  2.07313290e-03,   2.34984350e-03,   3.08062322e-03,
          2.57529248e-03,  -4.24361875e-04,  -4.47877275e-04,
          3.14351288e-03,   3.89899273e-04,  -1.993

In [77]:
f['Yields/Z_0.001/Total_Metals'][:]

array([  0.13855194,   0.40709192,   1.35447276,   1.01172721,
         0.83892071,   1.18154895,   4.02799034,   6.67571306,
         8.90166378,  12.46051788,   4.99824095,  11.14951515,   4.75875854], dtype=float32)

In [78]:
names = [n for n in f['Yields/Z_0.02'].keys()]

In [79]:
names

['Ejected_mass', 'Total_Metals', 'Yield']

## CC-SN YIELDS


In [267]:
indexing = {}
indexing['H'] = 'Hydrogen'
indexing['He'] = 'Helium'
indexing['C'] = 'Carbon'
indexing['N']= 'Nitrogen'
indexing['O'] = 'Oxygen'
indexing['Ne'] = 'Neon'
indexing['Mg'] = 'Magnesium'
indexing['Si'] = 'Silicon'
indexing['S'] = 'Sulphur' # Not used by TNG simulation
indexing['Ca'] = 'Calcium' # Not used by TNG simulation
indexing['Fe'] = 'Iron'

elements = list(indexing.keys())

table = {}

metallicities = list(f['Metallicities'].value)
masses = f['Masses'].value


for z_index,z in enumerate(metallicities):
    
    yield_subtable = {}
    
    z_name = f['Yield_names'].value[z_index].decode('utf-8')
    z_data = f['Yields/'+z_name+'/Yield']
     
    ejecta_mass = f['Yields/'+z_name+'/Ejected_mass'].value
    
    yield_subtable['Mass'] = masses
    remnants = masses-ejecta_mass
    yield_subtable['mass_in_remnants'] = np.divide(remnants,masses)
    for el in list(indexing.keys()):
        yield_subtable[el] = np.zeros(len(masses))
     
    summed_yields = np.zeros(len(masses))
        
    for m_index,mass in enumerate(masses):
        for el_index,el in enumerate(elements):
            el_yield = z_data[el_index][m_index]
            el_yield_fraction = el_yield/(mass-remnants[m_index])
            yield_subtable[el][m_index] = el_yield_fraction
            summed_yields[m_index]+=el_yield_fraction
         
    yield_subtable['unprocessed_mass_in_winds'] = 1.-summed_yields-yield_subtable['mass_in_remnants']
    
    
    table[z.astype(float)] = yield_subtable
    
    # Restructure table
    all_keys = ['Mass','mass_in_remnants','unprocessed_mass_in_winds']+elements

    list_of_arrays = [yield_subtable[key] for key in all_keys]
    restructure_subtable = np.core.records.fromarrays(list_of_arrays,names=all_keys)

    table[z] = restructure_subtable

        

In [268]:
table[metallicities[3]]

rec.array([ (   8.,  0.35942191,  0.64057808, -0.12350683,  0.1029522 ,  0.00032646,  0.00321482,  0.01195691,  0.00103193,  0.00114013,  0.00356276,  -6.92857648e-05,  -3.71730166e-05, -0.00057193),
 (   9.,  0.33912018,  0.66087982, -0.15501787,  0.10385986,  0.00233874,  0.00333841,  0.01976553,  0.00271937,  0.00102403,  0.00852475,   5.97119192e-03,   8.98001366e-04,  0.00657798),
 (  12.,  0.30720448,  0.69279552, -0.21484816,  0.08282538,  0.00722998,  0.00320906,  0.07619383,  0.01721282,  0.0040116 ,  0.00891201,   4.08016983e-03,   5.44621027e-04,  0.01062868),
 (  13.,  0.32168815,  0.67831188, -0.14169773,  0.09888977,  0.00231689,  0.00247367,  0.01482159,  0.00397462,  0.00409992,  0.00568473,   2.31677154e-03,   2.14743384e-04,  0.00690501),
 (  15.,  0.30451724,  0.69548277, -0.15058133,  0.12184691, -0.00104081,  0.0028229 ,  0.00606428,  0.00276856,  0.00486294,  0.0054797 ,   1.75553362e-03,   1.31738954e-04,  0.00588956),
 (  18.,  0.28380913,  0.71619087, -0.194821

In [19]:
f['Yields/Z_0.004/Yield'][1][5]/(12.588024)

0.15777491746376046

In [15]:
f['Yields/Z_0.004/Ejected_mass'][5]

12.588024

In [18]:
f['Masses'][5]

18.0

## AGB Yields

In [1]:
import h5py as h5
filename = 'AGB.hdf5'
# Read H5 file
f = h5.File(filename, "r")
# Get and print list of datasets within the H5 file
datasetNames = [n for n in f.keys()]
for n in datasetNames:
    print(n)

Masses
Metallicities
Number_of_masses
Number_of_metallicities
Number_of_species
Reference
Species_names
Yield_names
Yields


In [9]:
f['Species_names'].value
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/home/philcox/anaconda3/lib/python3.6/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['f', 'table']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [22]:
indexing = {}
indexing['H'] = 'Hydrogen'
indexing['He'] = 'Helium'
indexing['C'] = 'Carbon'
indexing['N']= 'Nitrogen'
indexing['O'] = 'Oxygen'
indexing['Ne'] = 'Neon'
indexing['Mg'] = 'Magnesium'
indexing['Si'] = 'Silicon'
indexing['S'] = 'Sulphur' # Not used by TNG simulation
indexing['Ca'] = 'Calcium' # Not used by TNG simulation
indexing['Fe'] = 'Iron'

elements = list(indexing.keys())

table = {}

metallicities = list(f['Metallicities'].value)
masses = f['Masses'].value


for z_index,z in enumerate(metallicities):
    
    yield_subtable = {}
    
    z_name = f['Yield_names'].value[z_index].decode('utf-8')
    z_data = f['Yields/'+z_name+'/Yield']
     
    ejecta_mass = f['Yields/'+z_name+'/Ejected_mass'].value
    
    yield_subtable['Mass'] = masses
    remnants = masses-ejecta_mass
    yield_subtable['mass_in_remnants'] = np.divide(remnants,masses)
    for el in list(indexing.keys()):
        yield_subtable[el] = np.zeros(len(masses))
     
    summed_yields = np.zeros(len(masses))
        
    for m_index,mass in enumerate(masses):
        for el_index,el in enumerate(elements):
            el_yield = z_data[el_index][m_index]
            el_yield_fraction = el_yield/mass
            yield_subtable[el][m_index] = el_yield_fraction
            summed_yields[m_index]+=el_yield_fraction
    
    yield_subtable['unprocessed_mass_in_winds'] = 1.-summed_yields-yield_subtable['mass_in_remnants']
    
    
    table[z.astype(float)] = yield_subtable
    
    # Restructure table
    all_keys = ['Mass','mass_in_remnants','unprocessed_mass_in_winds']+elements

    list_of_arrays = [yield_subtable[key] for key in all_keys]
    restructure_subtable = np.core.records.fromarrays(list_of_arrays,names=all_keys)

    table[z] = restructure_subtable

        

In [36]:
zs=f['Metallicities'].value

In [23]:
table

{9.9999997473787516e-05: rec.array([ ( 1.        ,  0.71999997,  0.27999944, -0.00564262,  0.00537587,   2.50535522e-04,   4.01939451e-06,   1.14432214e-05,   1.31620754e-06,   2.10177475e-08,   7.95601185e-09,  -1.94164934e-10,  0.,  -5.20256016e-09),
  ( 1.25      ,  0.53439969,  0.46560163, -0.02390677,  0.02078049,   2.97993259e-03,   1.58289695e-05,   7.63424978e-05,   5.26218100e-05,   2.48608586e-07,   1.66242504e-08,  -2.86256840e-09,  0.,  -1.38821949e-08),
  ( 1.5       ,  0.44066635,  0.55933733, -0.03916926,  0.03085452,   7.73600675e-03,   2.65822255e-05,   1.83819328e-04,   3.61945800e-04,   2.71054955e-06,   2.90254434e-08,  -5.02473574e-09,  0.,  -2.75934440e-08),
  ( 1.75      ,  0.37942901,  0.62059427, -0.06453545,  0.0498048 ,   1.32171763e-02,   3.06095390e-05,   2.44077077e-04,   1.19861611e-03,   1.68841761e-05,   6.70140849e-08,  -1.04325659e-08,  0.,  -5.22271293e-08),
  ( 1.89999998,  0.35842049,  0.64161971, -0.07412639,  0.05638204,   1.56162567e-02,   3.383

In [32]:
from Chempy.yields import AGB_feedback
kar = AGB_feedback()
getattr(kar,'Karakas_net_yield')()

In [35]:
kar.table[0.004][7]

(2.5, 0.2684, 0.731599004704226, -0.05999222, 0.044688133908, -1.11778792196e-09, 0.013545528595599998, 0.000233160167432, 3.87482464e-05, 3.4633270799999995e-06, 0.00142590401244, 2.5906309199999997e-05, 3.08197555256e-05, 1.16228404e-06, 5.11270992e-07, 4.4287888076e-08, -3.3417286299695316e-07, 1.51782528e-08, 1.7725942400000002e-07, -5.636402399999997e-07, 4.0299909320000003e-07, 1.3662529729073192e-07)

In [43]:
table[zs[1]][7]

(2.5, 0.2683998644351959, 0.7316302172930307, -0.05999220162630081, 0.04468813166022301, 0.013545568101108074, 0.0002331200521439314, 3.87482468795497e-05, 0.0014259039890021086, 3.080265742028132e-05, 5.112719350108819e-07, -3.189946085058182e-07, 0.0, -3.4708602925093146e-07)

In [18]:
list(reversed(f['Masses']))

[7.5,
 7.0,
 6.0,
 5.5,
 5.0,
 4.5,
 4.0,
 3.5,
 3.0,
 2.5,
 2.25,
 2.0,
 1.9,
 1.75,
 1.5,
 1.25,
 1.0]

In [3]:
f['Yields/Z_0.02/Yield'].value

array([[ -8.10482260e-03,  -1.22941164e-02,  -1.28319599e-02,
         -1.18061546e-02,  -1.08509110e-02,  -1.01878652e-02,
         -1.50535768e-02,  -3.00319660e-02,  -6.95260838e-02,
         -9.37633067e-02,  -6.30576909e-02,  -1.08568802e-01,
         -1.82101279e-01,  -2.75364280e-01,  -3.51533979e-01,
         -3.45729828e-01,  -4.02550012e-01],
       [  8.07338767e-03,   1.22192428e-02,   1.27050932e-02,
          1.16245942e-02,   1.06469290e-02,   9.96727683e-03,
          1.37838917e-02,   2.37319525e-02,   5.18163368e-02,
          6.98008016e-02,   4.90052029e-02,   9.45063233e-02,
          1.67598262e-01,   2.51472145e-01,   3.31333399e-01,
          3.45729947e-01,   4.02550012e-01],
       [ -1.50256485e-04,  -4.23887512e-04,  -7.45990605e-04,
         -1.06125791e-03,  -1.19475077e-03,  -1.28609734e-03,
         -5.67917887e-04,   3.90819646e-03,   1.38141243e-02,
          1.88335534e-02,   9.35914367e-03,   8.76018405e-03,
          5.20489085e-03,  -5.86592546e-03

In [11]:
f['Yields/Z_0.004/Yield'][-1][11]/4.5

-3.2757905298947462e-06

In [9]:
f['Masses'].value

array([ 1.        ,  1.25      ,  1.5       ,  1.75      ,  1.89999998,
        2.        ,  2.25      ,  2.5       ,  3.        ,  3.5       ,
        4.        ,  4.5       ,  5.        ,  5.5       ,  6.        ,
        7.        ,  7.5       ], dtype=float32)

In [16]:
-2.2355889E-06+-3.0700350E-05+ 1.2198179E-06+ 9.4808256E-06

-2.2235295399999996e-05

In [17]:
-2.2235295399999996e-05/4.5

-4.941176755555554e-06

## SNIa

In [53]:
import h5py as h5
filename = 'SNIa.hdf5'
# Read H5 file
f = h5.File(filename, "r")
# Get and print list of datasets within the H5 file
datasetNames = [n for n in f.keys()]
for n in datasetNames:
    print(n)

Number_of_species
Reference
Species_names
Total_Metals
Yield


In [55]:

np.sum(f['Yield'].value)

1.3743416

In [43]:

f['Total_Metals'].value

1.3740858

In [32]:
f['Reference'].value

b'Thielemann et al. 2003 (From Twilight to Highlight: The Physics of SN proceedings, ESO/MPA/MPE Workshop, pg 331) - as tabulated in Travaglio et al. 2004 (A&A, 425, 1029)'

In [81]:
import h5py as h5
filename = 'SNIa.hdf5'
# Read H5 file
f = h5.File(filename, "r")

indexing = {}
indexing['H'] = 'Hydrogen'
indexing['He'] = 'Helium'
indexing['Li'] = 'Lithium'
indexing['Be'] = 'Beryllium'
indexing['B'] = 'Boron'
indexing['C'] = 'Carbon'
indexing['N'] = 'Nitrogen'
indexing['O'] = 'Oxygen'
indexing['F'] = 'Fluorine'
indexing['Ne'] = 'Neon'
indexing['Na'] = 'Sodium'
indexing['Mg'] = 'Magnesium'
indexing['Al'] = 'Aluminum'
indexing['Si'] = 'Silicon'
indexing['P'] = 'Phosphorus'
indexing['S'] = 'Sulphur'
indexing['Cl'] = 'Chlorine'
indexing['Ar'] = 'Argon'
indexing['K'] = 'Potassium'
indexing['Ca'] = 'Calcium'
indexing['Sc'] = 'Scandium'
indexing['Ti'] = 'Titanium'
indexing['V'] = 'Vanadium'
indexing['Cr'] = 'Chromium'
indexing['Mn'] = 'Manganese'
indexing['Fe'] = 'Iron'
indexing['Co'] = 'Cobalt'
indexing['Ni'] = 'Nickel'
indexing['Cu'] = 'Copper'
indexing['Zn'] = 'Zinc'
indexing['Ga'] = 'Gallium'
indexing['Ge'] = 'Germanium'
indexing['As'] = 'Arsenic'
indexing['Se'] = 'Selenium'
indexing['Br'] = 'Bromine'
indexing['Kr'] = 'Krypton'
indexing['Rb'] = 'Rubidium'
indexing['Sr'] = 'Strontium'
indexing['Y'] = 'Yttrium'
indexing['Zr'] = 'Zirconium'
indexing['Nb'] = 'Niobium'
indexing['Mo'] = 'Molybdenum'


elements = list(indexing.keys())

table = {}

metallicities = list([0.02]) # arbitrary since only one value
masses = list([np.sum(f['Yield'].value)]) # sum of all yields

names = ['Mass','mass_in_remnants']+elements

yield_subtable = {}

base = np.zeros(len(masses))
list_of_arrays = []
for i in range(len(names)):
    list_of_arrays.append(base)
    
yield_subtable = np.core.records.fromarrays(list_of_arrays,names=names)

yield_subtable['Mass'] = masses
yield_subtable['mass_in_remnants'] = [-1*m for m in masses]

for el_index,el in enumerate(elements):
    yield_subtable[el] = np.divide(f['Yield'][el_index],masses)

table = {}
table[metallicities[0]] = yield_subtable

In [78]:
len(masses)

1

In [82]:
def Seitenzahl(self):
		"""
		Seitenzahl 2013 from Ivo txt
		"""
		y = np.genfromtxt(localpath + 'input/yields/Seitenzahl2013/0.02.txt', names = True, dtype = None)
		self.metallicities = list([0.02])
		self.masses = list([1.4004633930489443])
		names = list(y.dtype.names)
		self.elements = names[2:]
		base = np.zeros(len(self.masses))
		list_of_arrays = []
		for i in range(len(names)):
			list_of_arrays.append(base)
		yield_tables_final_structure_subtable = np.core.records.fromarrays(list_of_arrays,names=names)
		for name in names:
			if name in ['Mass','mass_in_remnants']:
				yield_tables_final_structure_subtable[name] = y[name]
			else:
				yield_tables_final_structure_subtable[name] = np.divide(y[name],self.masses)
		yield_tables_final_structure = {}
		yield_tables_final_structure[0.02] = yield_tables_final_structure_subtable
		self.table = yield_tables_final_structure
	

In [1]:
import h5py as h5
filename = 'SNII.hdf5'
# Read H5 file
f = h5.File(filename, "r")
# Get and print list of datasets within the H5 file
datasetNames = [n for n in f.keys()]
for n in datasetNames:
    print(n)

Masses
Metallicities
Number_of_masses
Number_of_metallicities
Number_of_species
Reference
Species_names
Yield_names
Yields


In [20]:
f['Yields/Z_0.02/Yield'][2]/f['Masses'].value

array([ 0.00020912,  0.00154562,  0.0050089 ,  0.00157157, -0.00072387,
        0.00112232,  0.00388577,  0.00178393,  0.00170952,  0.00620426,
        0.04526725,  0.06421332,  0.039109  ], dtype=float32)

In [5]:
f['Masses'].value

array([   8.,    9.,   12.,   13.,   15.,   18.,   20.,   25.,   30.,
         40.,   60.,  100.,  120.], dtype=float32)

In [19]:
import numpy as np
np.sum(f['Yields/Z_0.02/Yield'],axis=0)

array([  1.42026693e-08,   0.00000000e+00,  -2.01165676e-07,
        -1.22934580e-07,  -7.82310963e-08,  -1.11758709e-07,
        -5.21540642e-08,   4.09781933e-08,   3.20374966e-07,
         4.88013029e-07,  -4.02331352e-07,   5.96046448e-07,
         5.96046448e-07], dtype=float32)

In [21]:
0.108/13

0.008307692307692308

In [24]:
f['Species_names'][2]

b'Carbon'

In [26]:
f['Yields/Z_0.001/Yield'][2]/f['Masses']

array([ 0.00306853,  0.00564855,  0.00821065,  0.00451014,  0.00290938,
        0.00386448,  0.00331178,  0.00464376,  0.00181249,  0.00048273,
        0.00356878,  0.00222935,  0.00176894], dtype=float32)